# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [1]:
import string
import requests
from bs4 import BeautifulSoup
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 50
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Scraping page 1


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame()
df["reviews"] = reviews
df.head()

In [ ]:
df.to_csv("data/BA_reviews.csv")

Congratulations! Now you have your dataset for this task! The loops above collected 1000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [2]:
df = pd.read_csv("data/BA_reviews_3.csv")

In [3]:
analyzer = SentimentIntensityAnalyzer()
negative = []
neutral = []
positive = []
compound = []

In [4]:
for n in range(df.shape[0]):
    reviews = df.iloc[n,0]
    reviews_analyzed = analyzer.polarity_scores(reviews)
    negative.append(reviews_analyzed['neg'])
    neutral.append(reviews_analyzed['neu'])
    positive.append(reviews_analyzed['pos'])

    if reviews_analyzed['compound'] >= 0.05:
        compound.append("Positive")
    elif reviews_analyzed['compound'] <= -0.05:
        compound.append("Negative")
    else:
        compound.append("Neutral")

df['Negative'] = negative
df['Neutral'] = neutral
df['Positive'] = positive
df['Compound'] = compound
print(df)
    

TypeError: 'numpy.int64' object is not iterable

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df2.head()

,Unnamed: 0,reviews
0,0,Trip Verified | BA 242 on the 6/2/23. Boardin...
1,1,Trip Verified | Not only my first flight in ...
2,2,Trip Verified | My husband and myself were f...
3,3,Trip Verified | Organised boarding process. R...
4,4,Trip Verified | Outward journey BA245 London...


VS 2

In [10]:
df3 =df2.copy()

In [11]:
import re

def preprocess_reviews(reviews):
    # remove punctuation
    reviews = reviews.translate(str.maketrans('', '', string.punctuation))
    
    # convert to lowercase
    reviews = reviews.lower()

    # remove numbers
    reviews = re.sub(r'\d+', '', reviews)

    # remove special characters
    reviews = re.sub(r'[^\x00-\x7F]+', '', reviews)

    # remove whitespaces
    reviews = reviews.strip()

    return reviews


# Apply the preprocessing function to the reviews column
df3['reviews'] = df3['reviews'].apply(preprocess_reviews)

In [12]:
# remove unverified rows
df3 = df3[~df3['reviews'].str.contains("not verified", case=False) & ~df3['reviews'].str.contains("univerified", case=False)]

In [13]:
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_reviews(reviews):
    tokens = word_tokenize(reviews)
    return tokens


# Apply the tokenization function to the reviews column
df3['reviews'] = df3['reviews'].apply(tokenize_reviews)

In [14]:
from sklearn.preprocessing import LabelEncoder
# COnvert the target labels to numerical values
le = LabelEncoder()
y = le.fit_transform(df3['sentiment'])

# Join the list of tokens back into a single string
#df3['revi']

KeyError: 'sentiment'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create a bag of words representationof the review data
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
bow_representation = vectorizer.fit_transform(df3['reviews'])

In [ ]:
df3.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


#convert the target labels to numerical values
le = LabelEncoder()
y = le.fit_transform(df3['reviews'])
# spilt the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bow_representation, y, test_size=0.2)

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

#predict on the test data
y_pred = model.predict(X_test)

# calculate the accuracy
model_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", model_accuracy)